In [31]:
import pandas as pd
from pandas_datareader.data import DataReader
from datetime import datetime
import os

# Fred API
FRED_API_KEY = 'c5bd769e083c29db4135dfff57de42fd'
os.environ['FRED_API_KEY'] = FRED_API_KEY

def debug_fred_data(series_ids, start_date='1980-01-01', end_date='2020-12-31'):
    successful_data = {}
    failed_series = []

    for series_id in series_ids:
        try:
            data = DataReader(series_id, 'fred', start_date, end_date)
            data.rename(columns={series_id: series_id}, inplace=True)
            successful_data[series_id] = data
            print(f"✅ Successfully retrieved: {series_id} with {len(data)} rows.")
        except Exception as e:
            print(f"❌ Failed to retrieve: {series_id} — {e}")
            failed_series.append(series_id)

    return successful_data, failed_series

# Series IDs
series_ids = [
    'CPILFESL',      # Consumer Price Index (CPI)
    'UNRATE',        # Unemployment Rate
    'GDPC1',         # Real GDP
    'FEDFUNDS',      # Federal Funds Rate
    'DGS1',          # 1-Year Treasury Yield
    'DGS10',         # 10-Year Treasury Yield
    'M2SL',          # M2 Money Stock
    'DCOILWTICO',    # Crude Oil Prices (WTI)
    'CSUSHPINSA',    # Home Price Index
    'RSXFS',         # Retail Sales (Ex Autos)
    'UMCSENT'        # Consumer Sentiment Index
]

# Run
successful_data, failed_series = debug_fred_data(series_ids)

# success
for series_id, df in successful_data.items():
    print(f"\n📊 {series_id} Sample Data:")
    print(df.head())

# fail
if failed_series:
    print("\n The following series failed to load:")
    for s in failed_series:
        print(f"- {s}")
else:
    print("\n All series retrieved successfully!")


✅ Successfully retrieved: CPILFESL with 492 rows.
✅ Successfully retrieved: UNRATE with 492 rows.
✅ Successfully retrieved: GDPC1 with 164 rows.
✅ Successfully retrieved: FEDFUNDS with 492 rows.
✅ Successfully retrieved: DGS1 with 10698 rows.
✅ Successfully retrieved: DGS10 with 10698 rows.
✅ Successfully retrieved: M2SL with 492 rows.
✅ Successfully retrieved: DCOILWTICO with 9131 rows.
✅ Successfully retrieved: CSUSHPINSA with 408 rows.
✅ Successfully retrieved: RSXFS with 348 rows.
✅ Successfully retrieved: UMCSENT with 492 rows.

📊 CPILFESL Sample Data:
            CPILFESL
DATE                
1980-01-01      76.7
1980-02-01      77.5
1980-03-01      78.6
1980-04-01      79.5
1980-05-01      80.1

📊 UNRATE Sample Data:
            UNRATE
DATE              
1980-01-01     6.3
1980-02-01     6.3
1980-03-01     6.3
1980-04-01     6.9
1980-05-01     7.5

📊 GDPC1 Sample Data:
               GDPC1
DATE                
1980-01-01  7341.557
1980-04-01  7190.289
1980-07-01  7181.743
1980-1

In [30]:
def retrieve_and_merge_fred_data(series_ids, start_date='1980-01-01', end_date='2024-12-31'):
    merged_data = pd.DataFrame()

    for series_id in series_ids:
        try:
            data = DataReader(series_id, 'fred', start_date, end_date)
            data.rename(columns={series_id: series_id}, inplace=True)

            # Special handling for GDP (Quarterly Data)
            if series_id == 'GDPC1':
                data = data.resample('ME').ffill()  # Forward-fill quarterly data to monthly
                print(f"📊 {series_id} detected as Quarterly. Resampled to monthly.")
            else:
                data = data.resample('ME').mean()  # For monthly/daily/weekly data
                print(f"✅ {series_id} detected as Higher Frequency. Aggregated to monthly.")

            # Merge the data
            if merged_data.empty:
                merged_data = data
            else:
                merged_data = pd.merge(merged_data, data, left_index=True, right_index=True, how='outer')

        except Exception as e:
            print(f"❌ Failed to retrieve {series_id}: {e}")

    # Calculate Inflation Rate if CPI exists
    if 'CPILFESL' in merged_data.columns:
        merged_data['Inflation_Rate'] = merged_data['CPILFESL'].pct_change() * 100

    # Drop rows with NaN values caused by missing data
    merged_data.dropna(inplace=True)

    return merged_data

# Series IDs
series_ids = [
    'CPILFESL',      # Consumer Price Index (CPI) - Monthly
    'UNRATE',        # Unemployment Rate - Monthly
    'GDPC1',         # Real GDP - Quarterly
    'FEDFUNDS',      # Federal Funds Rate - Daily
    'DGS1',          # 1-Year Treasury Yield - Daily
    'DGS10',         # 10-Year Treasury Yield - Daily
    'M2SL',          # M2 Money Stock - Weekly
    'DCOILWTICO',    # Crude Oil Prices - Daily
    'CSUSHPINSA',    # Home Price Index - Monthly
    'RSXFS',         # Retail Sales - Monthly
    'UMCSENT'        # Consumer Sentiment Index - Monthly
]

# Retrieve and merge the data
merged_data = retrieve_and_merge_fred_data(series_ids)

# Count unique dates
unique_dates_count = merged_data.index.nunique()
print(f"📅 Number of unique dates: {unique_dates_count}")


#merged_data.to_csv('fred_macroeconomic_data_final.csv')


✅ CPILFESL detected as Higher Frequency. Aggregated to monthly.
✅ UNRATE detected as Higher Frequency. Aggregated to monthly.
📊 GDPC1 detected as Quarterly. Resampled to monthly.
✅ FEDFUNDS detected as Higher Frequency. Aggregated to monthly.
✅ DGS1 detected as Higher Frequency. Aggregated to monthly.
✅ DGS10 detected as Higher Frequency. Aggregated to monthly.
✅ M2SL detected as Higher Frequency. Aggregated to monthly.
✅ DCOILWTICO detected as Higher Frequency. Aggregated to monthly.
✅ CSUSHPINSA detected as Higher Frequency. Aggregated to monthly.
✅ RSXFS detected as Higher Frequency. Aggregated to monthly.
✅ UMCSENT detected as Higher Frequency. Aggregated to monthly.
📅 Number of unique dates: 394


In [32]:
# Detect any null values in the merged dataset
null_summary = merged_data.isnull().sum()
# Detect the columns with null values
null_columns = null_summary[null_summary > 0]
print(null_columns)

Series([], dtype: int64)
